In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
import random
               
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
fy_10 = pd.read_csv('fy_10_augmented.csv')
fy_11 = pd.read_csv('fy_11_augmented.csv')
fy_12 = pd.read_csv('fy_12_augmented.csv')
fy_13 = pd.read_csv('fy_13_augmented.csv')
fy_14 = pd.read_csv('fy_14_augmented.csv')
fy_15 = pd.read_csv('fy_15_augmented.csv')


In [3]:
fy_10.head(1)

,Unnamed: 0,Fund Id,Business Area,Fund Center Id,GL Account,GL Description,GL Category,GL Category Description,Revenue or Expenditure,Fiscal Year,Fund Name,Fund Type Descr,Business Area Name,Fund Center Name,Commitment Set Description,Original Budget,Current Budget,Actuals,3Y_Past_Actuals,2Y_Past_Actuals,1Y_Past_Actuals,2Y_1Y_Actuals_Mean,3Y_2Y_1Y_Actuals_Mean,3Y_Past_Original_Budget,2Y_Past_Original_Budget,1Y_Past_Original_Budget,2Y_1Y_Original_Budget_Mean,3Y_2Y_1Y_Original_Budget_Mean,3Y_Past_OA_Difference,2Y_Past_OA_Difference,1Y_Past_OA_Difference,2Y_1Y_OA_Difference_Mean,3Y_2Y_1Y_OA_Difference_Mean,OA_Difference,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Houston Police Department,Business Area Name_Human Resources Dept.,Business Area Name_Information Technology (IT),Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Court Judicial,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Yearly Crime Index,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Rain Fall,Astros Playoff Rounds,Astros Yearly Attendance,Rockets Playoff Rounds,Texans Playoff Rounds,Texans Average Attendance,Rockets Total Attendance,Business Area Name_Houston Police Department-HPD,Business Area Name_Houston Information Tech Svcs
0,0,1000,3800,3800050001,501070,501070 - Pension - Civilian,500,500 - Personnel Services,Expenditures,2010,General Fund,General Funds,Health & Human Services,HHS-Office Prog Supp,50 - Total Expenditures,113579.0,113579.0,121344.82,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,7765.82,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [7]:
drop_list = ['GL Description','GL Category Description', 'Revenue or Expenditure',
             'Unnamed: 0', 'Fund Name', 'Fund Type Descr', 'Business Area Name', 
             'Fund Center Name', 'Commitment Set Description']

train_list = [fy_12, fy_13, fy_14]
test_list = [fy_15]

X_train = pd.concat(train_list).drop(['OA_Difference','Actuals','Current Budget']+drop_list, axis=1)
X_test = pd.concat(test_list).drop(['OA_Difference', 'Actuals', 'Current Budget']+drop_list, axis = 1)
y_train = pd.concat(train_list)['Actuals']
y_test = pd.concat(test_list)['Actuals']
y_original_budget =  pd.concat(test_list)['Original Budget']

model = xgb.XGBRegressor()
model.fit(X_train,y_train)
print (model)

/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[17:31:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)


In [8]:
xgb_test_predictions = model.predict(X_test)
xgb_train_predictions = model.predict(X_train)

ValueError: feature_names mismatch: ['Fund Id', 'Business Area', 'Fund Center Id', 'GL Account', 'GL Category', 'Fiscal Year', 'Original Budget', '3Y_Past_Actuals', '2Y_Past_Actuals', '1Y_Past_Actuals', '2Y_1Y_Actuals_Mean', '3Y_2Y_1Y_Actuals_Mean', '3Y_Past_Original_Budget', '2Y_Past_Original_Budget', '1Y_Past_Original_Budget', '2Y_1Y_Original_Budget_Mean', '3Y_2Y_1Y_Original_Budget_Mean', '3Y_Past_OA_Difference', '2Y_Past_OA_Difference', '1Y_Past_OA_Difference', '2Y_1Y_OA_Difference_Mean', '3Y_2Y_1Y_OA_Difference_Mean', 'Fund Name_Asset Forfeiture Fund - Justice', 'Fund Name_Asset Forfeiture Fund - State', 'Fund Name_Auto Dealers Fund', 'Fund Name_Building Inspection Fund', 'Fund Name_C&E-Civic Center Facility Revenue Fund', 'Fund Name_Cable Television Fund', 'Fund Name_Child Safety Fund', 'Fund Name_Digital Houston Special Revenue', 'Fund Name_Expansion of Recycling Program', 'Fund Name_Fleet Management Fund', 'Fund Name_GSD - In-House Renovation Revolving Fund', 'Fund Name_General 

In [ ]:
np.sqrt(np.mean((xgb_test_predictions-y_test)**2))

In [ ]:
model_sum_error = np.sum(abs(xgb_test_predictions-y_test))

In [ ]:
budget_sum_error = np.sum(abs(y_original_budget-y_test))

### Model precision over budget

In [ ]:
print('$',round(budget_sum_error-model_sum_error,2))

In [ ]:
np.sqrt(np.mean((y_original_budget-y_test)**2))

In [ ]:
y_test.std()

In [ ]:
sns.set_style('whitegrid')

x = np.linspace(-1000000000,1000000000,1000000000)
plt.figure(figsize=(12,12))
plt.scatter(y_train, xgb_train_predictions, c='red', alpha=.95)
plt.scatter(y_test, xgb_test_predictions, c='blue', alpha=.95)
plt.scatter(y_test, y_original_budget, c='green', alpha=.95)
plt.plot(x,x,c='black')
plt.xlim([-5,1000000000])
plt.ylim([-5,1000000000])
plt.xlabel('observations')
plt.ylabel('predictions')


In [ ]:
weight_dict = model.get_booster().get_score(importance_type="weight")
weight_df = pd.DataFrame(weight_dict, index=range(len(weight_dict)))
weight_df.T[0].sort_values(ascending=False)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test , xgb_test_predictions)

In [ ]:

sns.set_style('darkgrid')
sns.set(font_scale=2.5)
bins = np.arange(0,90000,90000/50)
 
fig, ax = plt.subplots(figsize=(20,15))

sns.distplot(y_test, label='2015 Actuals',color='green', norm_hist=True, bins=bins, kde=False,  ax = ax)
sns.distplot(xgb_test_predictions, label='XGB_test_predictions 2015',color='black', norm_hist=True, bins=bins, kde=True,  ax = ax)
sns.distplot(y_original_budget, label='2015_original_budget',color='blue',norm_hist=True, bins=bins, kde=True,  ax = ax)
    
ax.set_title(" ")
ax.set_ylabel(" ")
ax.set_xlabel(" ")
ax.set_xlim(0,90000)
ax.legend()

In [ ]:
original_budget_residuals = fy_15['Original Budget']-fy_15['Actuals']
model_residuals = xgb_test_predictions-y_test
hold_df = pd.DataFrame()
hold_df['Original Budget Residuals'] = original_budget_residuals
hold_df['Model Residuals'] = model_residuals


In [ ]:
np.sqrt(np.mean(model_residuals**2))

In [ ]:
np.sqrt(np.mean(original_budget_residuals**2))

In [ ]:
model_residuals_series = pd.Series(model_residuals)

In [ ]:
original_budget_residuals_series =pd.Series(original_budget_residuals)

In [ ]:
residuals_df = pd.DataFrame()
residuals_df['Model_Residuals'] = model_residuals_series
residuals_df['Original_Budget_Residuals'] = original_budget_residuals_series

extreme_residuals = residuals_df.loc[(residuals_df['Model_Residuals']>=80000) | (residuals_df['Model_Residuals']<=-80000) | (residuals_df['Original_Budget_Residuals']>=80000) | (residuals_df['Original_Budget_Residuals']<=-80000) ] 
residuals_under_80K = residuals_df.loc[(residuals_df['Model_Residuals']<80000) & (residuals_df['Model_Residuals']>-80000) & (residuals_df['Original_Budget_Residuals']<=80000) & (residuals_df['Original_Budget_Residuals']>=-80000) ] 


In [ ]:
print('Model Extreme Resisduals:           ',int(np.sqrt(np.mean(extreme_residuals['Model_Residuals']**2))), '\nOriginal Budget Extreme Resisduals: ',  int(np.sqrt(np.mean(extreme_residuals['Original_Budget_Residuals']**2))))
print(' ')
print('Model Under 80K Resisduals:           ',int(np.sqrt(np.mean(residuals_under_80K['Model_Residuals']**2))), '\nOriginal Budget Under 80K Resisduals: ',  int(np.sqrt(np.mean(residuals_under_80K['Original_Budget_Residuals']**2))))


In [ ]:
sns.set_style('darkgrid')
sns.set(font_scale=2.5)
 
fig, ax = plt.subplots(figsize=(20,15))
bins_ = np.arange(-80000,80000,80000/50)

sns.distplot(model_residuals, label='Model Residuals', norm_hist=True, bins = bins_, color='green',   ax = ax)
sns.distplot(original_budget_residuals, label='Original Budget Residuals',bins=bins_, color='black',  ax = ax)
    
ax.set_title(" ")
ax.set_ylabel(" ")
ax.set_xlabel(" ")
ax.set_xlim(-80000,80000)
ax.legend()

# H2o

In [ ]:
h2o.init()

In [ ]:
# create h2o frame for training data
training_frame = pd.concat([X_train,y_train], axis=1)
training_frame.columns = list(training_frame.columns[:-1]) + ['Actuals']
training_frame = h2o.H2OFrame(training_frame)

In [ ]:
# create h2o frame for testing data
testing_frame = pd.concat([X_test,y_test], axis=1)
testing_frame.columns = list(testing_frame.columns[:-1]) + ['Actuals']
testing_frame = h2o.H2OFrame(testing_frame)

In [ ]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(None, 'Actuals', training_frame)

In [ ]:
y_test.std()

In [ ]:
predicted_actuals = aml.predict(h2o.H2OFrame(pd.concat([X_test, y_test], axis=1)))
np.sqrt(np.mean((predicted_actuals['predict'].as_data_frame()['predict'].values-y_test)**2))

In [ ]:
aml.leader

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_train, aml.predict(training_frame).as_data_frame()['predict'].values, c='red')
plt.scatter(y_test, aml.predict(testing_frame).as_data_frame()['predict'].values, c='blue')
#plt.scatter(train_df['voyage_duration'], train_df['eta_diff'], c='green')
plt.plot(x,x,c='black')
plt.xlim([-5,100000000])
plt.ylim([-5,100000000])
plt.xlabel('observations')
plt.ylabel('predictions')

In [ ]:
#9/4 4:42pm